In [ ]:
%load_ext lab_black

In [ ]:
from cartes.osm import Nominatim

t = Nominatim.search("Tristan da Cunha")
lat, lon = float(t.lat), float(t.lon)
t

In [ ]:
import cartopy.io.shapereader as shpreader
from geopandas import GeoDataFrame


countries_110 = GeoDataFrame.from_file(
    shpreader.natural_earth(
        resolution="110m",
        category="cultural",
        name="admin_0_countries",
    )
)
countries_50 = GeoDataFrame.from_file(
    shpreader.natural_earth(
        resolution="50m",
        category="cultural",
        name="admin_0_countries",
    )
)
countries_10 = GeoDataFrame.from_file(
    shpreader.natural_earth(
        resolution="10m",
        category="cultural",
        name="admin_0_countries",
    )
)

In [ ]:
from pyproj import Geod
from shapely.geometry import Polygon

geod = Geod(ellps="WGS84")


def intersects(g, distance, lat=lat, lon=lon):
    lon_, lat_, back_ = geod.fwd(
        lon * np.ones(360),
        lat * np.ones(360),
        np.arange(0, 360),
        distance * np.ones(360),
    )
    p = Polygon(list(zip(lon_, lat_)))
    a = g.intersects(p)

    return any(a), p

In [ ]:
def closest_country(g):

    res = list([*intersects(g, x), x] for x in np.linspace(0, 5000e3, 1001))
    polygon, distance = next(
        (polygon, distance) for test, polygon, distance in res if test
    )

    a = g.intersects(polygon)
    df = g.loc[a]
    return dict(
        geometry=polygon,
        distance=distance,
        intersection=df.assign(
            longitude=df.geometry.centroid.x,
            latitude=df.geometry.centroid.y,
        ),
    )

In [ ]:
closest_110 = closest_country(countries_110.query('NAME != "Saint Helena"'))
closest_50 = closest_country(countries_50.query('NAME != "Saint Helena"'))
closest_10 = closest_country(countries_10.query('NAME != "Saint Helena"'))

In [ ]:
closest_50["intersection"] = closest_50["intersection"].assign(NAME="South Georgia")

closest_10["intersection"] = (
    closest_10["intersection"]
    .assign(
        geometry=closest_10["intersection"].intersection(closest_10["geometry"]),
        NAME="Bouvetøya (N)",
    )
    .assign(
        longitude=lambda df: df.geometry.centroid.x,
        latitude=lambda df: df.geometry.centroid.y,
    )
)

In [ ]:
import altair as alt

closest = GeoDataFrame.from_records([closest_110, closest_50, closest_10]).assign(
    distance=lambda df: (df["distance"] * 1e-3).astype(int)
)

text_data = alt.Chart(
    pd.DataFrame.from_records([dict(latitude=lat, longitude=lon, name=t.display_name)])
).encode(alt.Latitude("latitude"), alt.Longitude("longitude"), alt.Text("name"))

neighbours = alt.Chart(
    pd.concat(list(closest["intersection"])).assign(
        distance=[
            int(closest_110["distance"] * 1e-3),
            int(closest_50["distance"] * 1e-3),
            int(closest_10["distance"] * 1e-3),
        ]
    )
).encode(
    alt.Latitude("latitude"),
    alt.Longitude("longitude"),
    alt.Text("NAME"),
)


chart = (
    alt.layer(
        alt.Chart(alt.sphere()).mark_geoshape(
            stroke="#bab0ac", fill="#9ecae9", opacity=0.4
        ),
        alt.Chart(g).mark_geoshape(stroke="#fff", fill="#bab0ac", opacity=0.6),
        alt.Chart(closest.drop(columns=["intersection"]))
        .mark_geoshape(fill="rgb(0, 0, 0, 0)", strokeWidth=1)
        .encode(alt.Stroke("distance:N", title="Distances (in km)")),
        text_data.mark_circle(color="black"),
        neighbours.mark_text(dy=15),
        neighbours.mark_circle(size=50)
        .encode(alt.Color("distance:N", legend=None))
        .transform_filter("datum.NAME != 'South Africa'"),
    )
    .project("orthographic", rotate=(-lon, -lat))
    .configure_text(font="Fira Sans", fontSize=12)
    .properties(
        width=400,
        height=400,
        title="Tristan da Cunha",
    )
    .configure_title(font="Fira Sans", fontSize=18, anchor="start")
    .configure_legend(
        labelFont="Fira Sans",
        labelFontSize=13,
        titleFont="Fira Sans",
        titleFontSize=15,
        orient="bottom",
    )
    .configure_view(stroke=None)
)
chart

In [ ]:
chart.save("../contributions/challenge_day18.svg")